In [2]:
import pandas as pd
import sys
sys.path.insert(1, '../../../../scripts/')
from s3_support import *

In [22]:
q = '''select
            date_trunc('day', date) as day,
            form,
            sum(views) as pageviews
        from googleanalytics_traffic
        group by date_trunc('day', date), org, form;'''
pageviews = redshift_query_read(q)
pageviews = pageviews[pageviews['form']!=0]

In [28]:
q = '''select 
            form, 
            date_trunc('day', date) as day,
            count(id) as count, 
            sum(amount) as vol
        from transactions
        where status='A' and date>=2017 and source='don_form' and recurring=0
        group by form, date_trunc('day', date)'''
trans = redshift_query_read(q)

In [33]:
# merging & storing data
df = pageviews.merge(trans, on=['form', 'day'], how='left').fillna(0)
df['conversion'] = df['count'] / df['pageviews']
df.sort_values('day', ascending=True, inplace=True)

In [34]:
df.head()

,day,form,pageviews,count,vol,conversion
1612886,2017-02-11,851031,8,0.0,0.0,0.0
1382977,2017-02-11,858198,2,0.0,0.0,0.0
1596105,2017-02-11,55,1,0.0,0.0,0.0
1422789,2017-02-11,872555,6,0.0,0.0,0.0
3076795,2017-02-11,932791,7,0.0,0.0,0.0


In [35]:
len(df), len(df[df['pageviews']==0])

(3085576, 0)

In [50]:
grpd = df.groupby('form')[['pageviews', 'count', 'vol', 'conversion']].transform(lambda x: x.rolling(90, 1).mean()).reset_index()
grpd.head()

,index,pageviews,count,vol,conversion
0,1612886,8.0,0.0,0.0,0.0
1,1382977,2.0,0.0,0.0,0.0
2,1596105,1.0,0.0,0.0,0.0
3,1422789,6.0,0.0,0.0,0.0
4,3076795,7.0,0.0,0.0,0.0


In [48]:
df[df['form']==55][['pageviews', 'count', 'vol', 'conversion']].mean()

pageviews     12.432727
count          0.960000
vol           23.043636
conversion     0.057670
dtype: float64

In [51]:
grpd['form'] = grpd['index']
grpd.drop('index', axis=1, inplace=True)
grpd.head()

,pageviews,count,vol,conversion,form
0,8.0,0.0,0.0,0.0,1612886
1,2.0,0.0,0.0,0.0,1382977
2,1.0,0.0,0.0,0.0,1596105
3,6.0,0.0,0.0,0.0,1422789
4,7.0,0.0,0.0,0.0,3076795


In [52]:
grpd.tail()

,pageviews,count,vol,conversion,form
3085571,5.540230,0.000000,0.000000,0.000000,2943997
3085572,48.133333,3.266667,178.302222,0.140137,2943994
3085573,13.936508,0.000000,0.000000,0.000000,1118554
3085574,13.233333,0.144444,18.685111,0.001254,1105210
3085575,19.844444,0.088889,16.944444,0.004495,1097389
